In [ ]:
# Import Library
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from google.colab import drive
from google.colab import files

In [ ]:
#Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Setup data dari kaggle
!pip install kaggle
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle/'
files.upload()  # upload kaggle.json
!mkdir -p /content/drive/MyDrive/kaggle
!cp kaggle.json /content/drive/MyDrive/kaggle/
!chmod 600 /content/drive/MyDrive/kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
# Download data dari kaggle ke gdrive
!kaggle datasets download -d moltean/fruits -p /content/drive/MyDrive/kaggle


Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0
100% 3.46G/3.47G [00:28<00:00, 128MB/s]
100% 3.47G/3.47G [00:28<00:00, 130MB/s]


In [ ]:
# Jika sudah ada datanya copy ke runtime
!cp /content/drive/MyDrive/kaggle/fruits.zip /content/



In [ ]:
#unzip
!unzip -q /content/fruits.zip -d /content/fruits-360


In [ ]:
#tentukan train dan test directory
train_dir = '/content/fruits-360/fruits-360_100x100/fruits-360/Training'
test_dir = '/content/fruits-360/fruits-360_100x100/fruits-360/Test'

In [ ]:
#load data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical'
)


Found 103993 images belonging to 206 classes.
Found 34711 images belonging to 206 classes.


In [ ]:
# === BANGUN MODEL DENGAN CONV2D ===
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(train_generator.num_classes, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Path untuk menyimpan model
checkpoint_path = "/content/drive/MyDrive/model_checkpoint/fruit_model_epoch_{epoch:02d}_valacc_{val_accuracy:.2f}.h5"

# Callback untuk menyimpan model tiap epoch
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',      # Pantau akurasi validasi
    save_best_only=True,         # Simpan hanya model terbaik
    save_weights_only=False,     # Simpan model lengkap (struktur + bobot)
    verbose=1
)

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[checkpoint_callback]  # Tambahkan di sini
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3250/3250 ━━━━━━━━━━━━━━━━━━━━ 0s 745ms/step - accuracy: 0.2588 - loss: 3.0604
Epoch 1: val_accuracy improved from -inf to 0.83933, saving model to /content/drive/MyDrive/model_checkpoint/fruit_model_epoch_01_valacc_0.84.h5


3250/3250 ━━━━━━━━━━━━━━━━━━━━ 2618s 805ms/step - accuracy: 0.2589 - loss: 3.0601 - val_accuracy: 0.8393 - val_loss: 0.5187
Epoch 2/10
 404/3250 ━━━━━━━━━━━━━━━━━━━━ 35:04 740ms/step - accuracy: 0.7060 - loss: 0.8699

In [ ]:
# (OPSIONAL) Jika ingin melanjutkan training model dari checkpoint
# Load model dari file .h5
model = load_model("/content/drive/MyDrive/model_checkpoint/fruit_model_epoch_02_valacc_0.97.h5")
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# Lanjutkan training
history = model.fit(
    train_generator,
    epochs=8,
    validation_data=test_generator,
    callbacks=[checkpoint_callback]
)

In [ ]:
# Opsional Visualisasikan akurasi dan los
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()
plt.show()


In [ ]:
from tensorflow.keras.preprocessing import image
model = load_model("/content/drive/MyDrive/model_checkpoint/fruit_model_epoch_06_valacc_0.93.h5")

uploaded = files.upload()

# Ambil nama file yang diupload
img_path = list(uploaded.keys())[0]

# Load & preprocess gambar
img = image.load_img(img_path, target_size=(100, 100))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Prediksi
pred = model.predict(img_array)
class_idx = np.argmax(pred)
labels = list(train_generator.class_indices.keys())

# Tampilkan hasil
plt.imshow(img)
plt.axis('off')
plt.title(f"Predicted: {labels[class_idx]}")
plt.show()

print("Predicted class:", labels[class_idx])
os.remove(img_path)


NameError: name 'load_model' is not defined